In [21]:
import numpy as np
import pandas as pd
import re



In [131]:
#importing all the modules needed to make the naive bayes and clean the data
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from nltk.stem.porter import PorterStemmer
import os
from string import punctuation

In [132]:
#read in the data
jeopardy = pd.read_json('jeopardy.json')

In [133]:
#get a good look at some of the data
jeopardy.head()

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680


In [134]:
jeopardy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216930 entries, 0 to 216929
Data columns (total 7 columns):
category       216930 non-null object
air_date       216930 non-null object
question       216930 non-null object
value          213296 non-null object
answer         216930 non-null object
round          216930 non-null object
show_number    216930 non-null int64
dtypes: int64(1), object(6)
memory usage: 11.6+ MB


In [135]:
#i am making another df column that copies the value so I can use this to set value as High or low
jeopardy['value_char'] = jeopardy['value']

In [136]:
#Not the best way, but I use this for other applications. replacing a value with category. Would like to use <,>, but types not able to be converted at this point
jeopardy['value_char'].loc[jeopardy['value'] =='$600'] = 'High'
jeopardy['value_char'].loc[jeopardy['value'] == '$2000'] = 'High'
jeopardy['value_char'].loc[jeopardy['value'] == '$200'] = 'Low'
jeopardy['value_char'].loc[jeopardy['value'] =='$400'] = 'Low'
jeopardy['value_char'].loc[jeopardy['value'] == '$800'] = 'High'
jeopardy['value_char'].loc[jeopardy['value'] == '$1000'] = 'High'

In [137]:
#wanted to get a view at some other mid level values
jeopardy[18000:]

,category,air_date,question,value,answer,round,show_number,value_char
18000,BIRDS,1986-04-23,"'According to Aesop, greed killed the goose th...",$200,laid the golden egg,Double Jeopardy!,423,Low
18001,BASEBALL,1986-04-23,"'In 1919-21, this championship was a best-of-9...",$200,the World Series,Double Jeopardy!,423,Low
18002,THE 19TH CENTURY,1986-04-23,"'Great Britain, Prussia, Austria & Russia join...",$200,Napoleon,Double Jeopardy!,423,Low
18003,ASTRONOMY,1986-04-23,"'Of lunar, or solar eclipses, the more frequent'",$200,a solar eclipse,Double Jeopardy!,423,Low
18004,POTENT QUOTABLES,1986-04-23,"'In the pirate drinking song, it follows “yo-h...",$200,a bottle of rum,Double Jeopardy!,423,Low
...,...,...,...,...,...,...,...,...
216925,RIDDLE ME THIS,2006-05-11,'This Puccini opera turns on the solution to 3...,$2000,Turandot,Double Jeopardy!,4999,High
216926,"""T"" BIRDS",2006-05-11,'In North America this term is properly applie...,$2000,a titmouse,Double Jeopardy!,4999,High
216927,AUTHORS IN THEIR YOUTH,2006-05-11,"'In Penny Lane, where this ""Hellraiser"" grew u...",$2000,Clive Barker,Double Jeopardy!,4999,High
216928,QUOTATIONS,2006-05-11,"'From Ft. Sill, Okla. he made the plea, Arizon...",$2000,Geronimo,Double Jeopardy!,4999,High


In [138]:
jeopardy['value_char']

0          Low
1          Low
2          Low
3          Low
4          Low
          ... 
216925    High
216926    High
216927    High
216928    High
216929    None
Name: value_char, Length: 216930, dtype: object

In [139]:
#checking, 
jeopardy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216930 entries, 0 to 216929
Data columns (total 8 columns):
category       216930 non-null object
air_date       216930 non-null object
question       216930 non-null object
value          213296 non-null object
answer         216930 non-null object
round          216930 non-null object
show_number    216930 non-null int64
value_char     213296 non-null object
dtypes: int64(1), object(7)
memory usage: 13.2+ MB


In [140]:
#literally had so much messing around that I made a new df named jeo. Literally the same thing as above jeopardy. So I will rerun to have all code here
jeo = jeopardy


In [141]:
#want to see the unique values in the column
jeo['value'].unique()

array(['$200', '$400', '$600', '$800', '$2,000', '$1000', '$1200',
       '$1600', '$2000', '$3,200', None, '$5,000', '$100', '$300', '$500',
       '$1,000', '$1,500', '$1,200', '$4,800', '$1,800', '$1,100',
       '$2,200', '$3,400', '$3,000', '$4,000', '$1,600', '$6,800',
       '$1,900', '$3,100', '$700', '$1,400', '$2,800', '$8,000', '$6,000',
       '$2,400', '$12,000', '$3,800', '$2,500', '$6,200', '$10,000',
       '$7,000', '$1,492', '$7,400', '$1,300', '$7,200', '$2,600',
       '$3,300', '$5,400', '$4,500', '$2,100', '$900', '$3,600', '$2,127',
       '$367', '$4,400', '$3,500', '$2,900', '$3,900', '$4,100', '$4,600',
       '$10,800', '$2,300', '$5,600', '$1,111', '$8,200', '$5,800',
       '$750', '$7,500', '$1,700', '$9,000', '$6,100', '$1,020', '$4,700',
       '$2,021', '$5,200', '$3,389', '$4,200', '$5', '$2,001', '$1,263',
       '$4,637', '$3,201', '$6,600', '$3,700', '$2,990', '$5,500',
       '$14,000', '$2,700', '$6,400', '$350', '$8,600', '$6,300', '$250',
      

In [142]:
#drop the null values (none)
jeo=jeo.dropna()

In [143]:
jeo

,category,air_date,question,value,answer,round,show_number,value_char
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680,Low
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680,Low
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680,Low
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680,Low
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680,Low
...,...,...,...,...,...,...,...,...
216924,OFF-BROADWAY,2006-05-11,'In 2006 the cast of this long-running hit emb...,$2000,Stomp,Double Jeopardy!,4999,High
216925,RIDDLE ME THIS,2006-05-11,'This Puccini opera turns on the solution to 3...,$2000,Turandot,Double Jeopardy!,4999,High
216926,"""T"" BIRDS",2006-05-11,'In North America this term is properly applie...,$2000,a titmouse,Double Jeopardy!,4999,High
216927,AUTHORS IN THEIR YOUTH,2006-05-11,"'In Penny Lane, where this ""Hellraiser"" grew u...",$2000,Clive Barker,Double Jeopardy!,4999,High


In [144]:
#to make sure we only have high and low left
jeo['value_char'].unique()

array(['Low', 'High', '$2,000', '$1200', '$1600', '$3,200', '$5,000',
       '$100', '$300', '$500', '$1,000', '$1,500', '$1,200', '$4,800',
       '$1,800', '$1,100', '$2,200', '$3,400', '$3,000', '$4,000',
       '$1,600', '$6,800', '$1,900', '$3,100', '$700', '$1,400', '$2,800',
       '$8,000', '$6,000', '$2,400', '$12,000', '$3,800', '$2,500',
       '$6,200', '$10,000', '$7,000', '$1,492', '$7,400', '$1,300',
       '$7,200', '$2,600', '$3,300', '$5,400', '$4,500', '$2,100', '$900',
       '$3,600', '$2,127', '$367', '$4,400', '$3,500', '$2,900', '$3,900',
       '$4,100', '$4,600', '$10,800', '$2,300', '$5,600', '$1,111',
       '$8,200', '$5,800', '$750', '$7,500', '$1,700', '$9,000', '$6,100',
       '$1,020', '$4,700', '$2,021', '$5,200', '$3,389', '$4,200', '$5',
       '$2,001', '$1,263', '$4,637', '$3,201', '$6,600', '$3,700',
       '$2,990', '$5,500', '$14,000', '$2,700', '$6,400', '$350',
       '$8,600', '$6,300', '$250', '$3,989', '$8,917', '$9,500', '$1,246',
       

In [146]:
#Now i want to split my data to train and test sets. I am going to put the test size at 1/3
X_train, X_test, y_train, y_test = train_test_split(jeo.question, jeo.value_char, test_size=0.33, random_state=1)
Tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_tf = Tfidf_vectorizer.fit_transform(X_train)
X_test_tf = Tfidf_vectorizer.transform(X_test)

In [148]:
#I want to run it through a NB classifier to see if it can predict if its a high or low value question
naive_bayes=MultinomialNB()
naive_bayes.fit(X_train_tf, y_train)
predictions = naive_bayes.predict(X_test_tf)
print('accuracy:', accuracy_score(y_test,predictions))

accuracy: 0.40624822412911293
